Однонаправленная LSTM

Импортируем необходимые зависимости

In [1]:
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Загрузим текст для обучения из txt файла

In [2]:
with open('harry_potter.txt', 'r') as file:
    text = file.read()

Создадим токенизатор и словарь слов

In [14]:
# tokens = list(filter(None, [re.sub('[^а-яА-ЯёЁ0-9 ,-]', '', s).strip() for s in text.split('.')]))
tokens = word_tokenize(text)

word_dict = {}
for word in tokens:
    if word not in word_dict:
        word_dict[word] = len(word_dict) + 1

reverse_word_dict = {v: k for k, v in word_dict.items()}
print(reverse_word_dict)

sequences = [word_dict[word] for word in tokens]

{1: 'В', 2: 'лунном', 3: 'свете', 4: 'блестит', 5: 'полоска', 6: 'серебра…', 7: '(', 8: 'падают', 9: 'тёмные', 10: 'одежды', 11: ')', 12: '...', 13: 'кровь', 14: 'льётся', 15: 'литрами', 16: ',', 17: 'и', 18: 'слышен', 19: 'крик', 20: '.', 21: 'Все', 22: 'стены', 23: 'до', 24: 'последнего', 25: 'дюйма', 26: 'заняты', 27: 'книжными', 28: 'шкафами', 29: 'каждом', 30: '—', 31: 'по', 32: 'шесть', 33: 'полок', 34: 'шкафы', 35: 'почти', 36: 'доходят', 37: 'потолка', 38: 'Часть', 39: 'плотно', 40: 'заставлена', 41: 'книгами', 42: 'в', 43: 'твёрдом', 44: 'переплёте', 45: ':', 46: 'математика', 47: 'химия', 48: 'история', 49: 'так', 50: 'далее', 51: 'На', 52: 'других', 53: 'полках', 54: 'два', 55: 'ряда', 56: 'стоит', 57: 'научная', 58: 'фантастика', 59: 'мягкой', 60: 'обложке', 61: 'Под', 62: 'второй', 63: 'ряд', 64: 'книг', 65: 'подложены', 66: 'коробки', 67: 'деревянные', 68: 'бруски', 69: 'что', 70: 'он', 71: 'возвышается', 72: 'над', 73: 'первым', 74: 'можно', 75: 'прочитать', 76: 'названи

Создадим обучающие примеры

In [15]:
X = []
y = []
sequence_length = 50

for i in range(len(sequences) - sequence_length):
    X.append(sequences[i:i + sequence_length])
    y.append(sequences[i + sequence_length])

X = np.array(X)
y = to_categorical(y, num_classes=len(word_dict) + 1)

Создадим и обучим модель

In [16]:
embedding_dim = 50
hidden_units = 128

model = Sequential()
model.add(Embedding(input_dim=len(word_dict) + 1, output_dim=embedding_dim, input_length=sequence_length))
model.add(LSTM(hidden_units))
model.add(Dense(len(word_dict) + 1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
76/76 [==============================] - 9s 93ms/step - loss: 7.2132 - accuracy: 0.0863
Epoch 2/100
76/76 [==============================] - 7s 93ms/step - loss: 6.5798 - accuracy: 0.0894
Epoch 3/100
76/76 [==============================] - 7s 94ms/step - loss: 6.4491 - accuracy: 0.0894
Epoch 4/100
76/76 [==============================] - 7s 94ms/step - loss: 6.3301 - accuracy: 0.0896
Epoch 5/100
76/76 [==============================] - 7s 93ms/step - loss: 6.2182 - accuracy: 0.1095
Epoch 6/100
76/76 [==============================] - 7s 94ms/step - loss: 6.1194 - accuracy: 0.1242
Epoch 7/100
76/76 [==============================] - 7s 94ms/step - loss: 6.0368 - accuracy: 0.1255
Epoch 8/100
76/76 [==============================] - 7s 92ms/step - loss: 5.9654 - accuracy: 0.1261
Epoch 9/100
76/76 [==============================] - 7s 92ms/step - loss: 5.8922 - accuracy: 0.1265
Epoch 10/100
76/76 [==============================] - 7s 93ms/step - loss: 5.8124 - accuracy: 0.1278

Попробуем сгенерировать текст

In [18]:
def generate_text(seed_text):
    seed_tokens = word_tokenize(seed_text)
    seed_sequence = [word_dict[word] for word in seed_tokens]

    for _ in range(50):
        input_sequence = pad_sequences([seed_sequence], maxlen=sequence_length)
        predicted_probs = model.predict(input_sequence)[0]
        predicted_word_index = np.random.choice(np.argsort(predicted_probs)[-5:][::-1])
        seed_sequence.append(predicted_word_index)
        seed_sequence = seed_sequence[-sequence_length:]

    return seed_text + ' ' + ' '.join([reverse_word_dict[index] for index in seed_sequence])

In [19]:
seed_text = 'Магия'
generated_text = generate_text(seed_text)

print(generated_text)

1/1 [==============================] - 0s 28ms/step
Магия , обменять сикли некоторые ) ) ) ( о поведение карандаша ( всякий чаще вокруг на прилавках постоянно глаз ) ) , чтобы показать бы не возникло . Гарри отвернулся размером на взять ещё другое едкое золота на серебро , фигура дубликатом . * * * он был способен замечать
